In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn import set_config
from time import time
from datetime import datetime
from dateutil import parser
from sklearn.model_selection import cross_validate, ParameterGrid, GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder, LabelBinarizer
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import make_scorer, accuracy_score, classification_report

In [2]:
def prep_df(csv):
    global df_original
    df_original = pd.read_csv(csv)
    df = pd.read_csv(csv)
    df["date_today"] = datetime.now()
    df = df.dropna(subset = ["original_estimate_complete_datetime"])
    df["estimate_date_parse"] = df["original_estimate_complete_datetime"].apply(lambda x: parser.parse(str(x)))
    df = df.assign(time_since_estimate =  df['date_today'] - df['estimate_date_parse'])
    df["days_since_estimate"] = df["time_since_estimate"].apply(lambda x: int(str(x).split()[0]))
    global df_w_days
    df_w_days = df
    del df["time_since_estimate"]
    df1 = df[df["suppCnt"] == 0]
    df2 = df[df["suppCnt"] != 0]
    df1["suppInd"] = 0
    df2["suppInd"] = 1
    df = pd.concat([df1,df2], axis = 0)
    df["paid_loss_amount"] = df["paid_loss_amount"].apply(lambda x: abs(x))
    df["primary_point_of_impact"].fillna("Unknown",inplace = True)
    df["total_loss_points"].fillna(999,inplace = True)
    df["insured_fault_pct"].fillna(999,inplace = True)
    
    df = df[["claim_number","company","coverage_grouping","detailed_loss_cause",'driveable_ind',"insured_fault_pct","insured_fault_rating","line_of_business","loss_cause","loss_party_type","loss_state","original_estimate_complete_datetime","payAfterCloseInd","policy_state","policy_type","primary_point_of_impact","saved_vehicle_as_type","secondary_point_of_impact","total_loss_points","vehicle_make","vehicle_year","suppInd","maxNet","minNet","maxFile","minFile","days_since_estimate"]]
    global df_full_data
    df_full_data = df
    df = df[df["days_since_estimate"] >= 90]
    df = df.set_index("claim_number")
    return df

In [3]:
time1 = time()
print("Before beginninng, make sure that your CSV is in your juypter notebook folder.")
print("If it is not, kill the kernel now.")
csv_name = input("Please enter the CSV name: ")
new_csv = input("Please name output CSV: ")
df = prep_df(csv_name + ".csv")

Before beginninng, make sure that your CSV is in your juypter notebook folder.
If it is not, kill the kernel now.
Please enter the CSV name: Supp Prediction
Please name output CSV: Flagged Claims


C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17904\1980049728.py:3: DtypeWarning: Columns (28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv(csv)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17904\1980049728.py:4: DtypeWarning: Columns (28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17904\1980049728.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["estimate_date_parse"] = df["original_estimate_complete_datetime"].apply(lambda x: parser.parse(str(x)))
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17904\1980049728.py:15: SettingWithCopyWarning: 
A value is trying to be set on 

In [4]:
target_name = "suppInd"
Y = df[target_name]
X = df.drop(columns=[target_name,"payAfterCloseInd","days_since_estimate","maxNet","minNet","company","line_of_business","policy_state",])

In [5]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])

In [6]:
model = make_pipeline(preprocessor, DecisionTreeClassifier(max_depth = 7))
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42, train_size = 0.75)
clf = model.fit(X_train, Y_train)
accuracy = clf.score(X_test, Y_test)
df_under_90 = df_full_data[df_full_data["days_since_estimate"] < 90]
df_under_90["prediction"] = clf.predict(df_under_90)
predict_percent_90 = len(df_under_90[df_under_90["prediction"] == 1]) / len(df_under_90)
total_percent_supp = len(df_under_90[df_under_90["suppInd"] == 1][df_under_90["prediction"] == 1])/ len(df_under_90)
sup_over_90 = len(df[df["suppInd"] == 1])/len(df)
flagged_claims = df_under_90[df_under_90["prediction"] == 1][df_under_90["suppInd"] == 0][["claim_number"]]
report = "Accuracy of Data: " + str(accuracy) + "\n" + "Percent of claims that have supplements over 90 days: " + str(sup_over_90) + "\n" + "Percent of claims that have predicted supplements under 90 days: " +  str(predict_percent_90)  

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17904\1426249978.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_under_90["prediction"] = clf.predict(df_under_90)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17904\1426249978.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  total_percent_supp = len(df_under_90[df_under_90["suppInd"] == 1][df_under_90["prediction"] == 1])/ len(df_under_90)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_17904\1426249978.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  flagged_claims = df_under_90[df_under_90["prediction"] == 1][df_under_90["suppInd"] == 0][["claim_number"]]


In [7]:
flagged_claims_full_data = pd.merge(flagged_claims, df_w_days, how = "left", on = "claim_number").drop_duplicates().reset_index()
flagged_claims_full_data.to_csv((new_csv + '.csv'), index=False, encoding='utf-8')  

In [8]:
new_supp_count = len(flagged_claims_full_data)
print( "Count of claims that have predicted new supplements: " + str(new_supp_count))
time2 = time()
print("Runtime: " + str(round(time2 - time1, 3)) + " seconds")

Count of claims that have predicted new supplements: 13392
Runtime: 123.031 seconds


In [9]:
print(report)
print( "Count of claims that have predicted new supplements: " + str(new_supp_count))
print("Runtime: " + str(round(time2 - time1, 3)) + " seconds")
flagged_claims_full_data

Accuracy of Data: 0.8452982214798074
Percent of claims that have supplements over 90 days: 0.5369808094802936
Percent of claims that have predicted supplements under 90 days: 0.38846911759372316
Count of claims that have predicted new supplements: 13392
Runtime: 123.031 seconds


,index,claim_number,airbags_deployed_ind,all_over_damage_ind,claim_status,company,coverage_grouping,detailed_loss_cause,driveable_ind,early_tow_ind,...,total_loss_points,vehicle_condition_type,vehicle_make,vehicle_model,vehicle_year,vin,zeroRowValidation,date_today,estimate_date_parse,days_since_estimate
0,0,01-004-970117,1.0,0,Open,Connect,Collision,Front End Collision,No,1.0,...,34.0,NaN,Jeep,Compass 4d 4wd (New),2020,3C4NJDCB0LT116037,0,2022-08-16 09:15:08.088030,2022-07-15 15:44:24,31
1,1,01-004-933214,1.0,1,Closed,Amfam,Comprehensive,Hit Deer,No,NaN,...,41.0,NaN,Ford Truck,Explorer 4d 4x4,2020,1FMSK8DH3LGB81104,0,2022-08-16 09:15:08.088030,2022-07-12 15:46:45,34
2,2,01-004-753826,1.0,0,Closed,Connect,Collision,Sideswipe,No,1.0,...,35.0,NaN,Kia,Niro Hybrid 5d,2019,KNDCB3LCXK5257574,0,2022-08-16 09:15:08.088030,2022-05-27 08:41:34,81
3,3,01-004-758484,1.0,0,Closed,Amfam,Collision,Insured Rear-ended Claimant,No,NaN,...,44.0,NaN,Ford Truck,F150 Crew C Pu 4x4,2013,1FTFW1ET4DKD37024,0,2022-08-16 09:15:08.088030,2022-05-26 08:21:52,82
4,4,01-004-492433,1.0,0,Open,Amfam,Property Damage Liability,Collision While Turning,No,NaN,...,50.0,NaN,Dodge Truck,Durango 4d 4wd,2007,1d8hb58p77f583980,0,2022-08-16 09:15:08.088030,2022-06-03 11:41:02,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13387,14833,01-004-854038,NaN,0,Closed,Connect,Comprehensive,Theft of Vehicle/Watercraft Parts,No,NaN,...,41.0,NaN,Honda,Cr-V 4d 4wd,2003,JHLRD78823C049849,0,2022-08-16 09:15:08.088030,2022-06-21 07:07:27,56
13388,14834,01-004-688060,NaN,0,Closed,Amfam,Comprehensive,NaN,Yes,NaN,...,NaN,NaN,Ford Truck,Escape 4d 4wd,2017,1FMCU9G92HUB29609,0,2022-08-16 09:15:08.088030,2022-05-25 19:22:19,82
13389,14835,01-004-827913,NaN,0,Closed,Amfam,Comprehensive,NaN,Yes,NaN,...,NaN,NaN,Ford Truck,Ranger Crew Pu 4x4,2022,1FTER4FH4NLD05152,0,2022-08-16 09:15:08.088030,2022-07-12 15:48:33,34
13390,14836,01-004-723980,NaN,0,Closed,Amfam,Comprehensive,NaN,Yes,NaN,...,NaN,NaN,Jeep,Compass 4d 4wd (New),2021,3C4NJDBB2MT572074,0,2022-08-16 09:15:08.088030,2022-05-30 14:01:27,77
